In [1]:
import torch
# Mengimpor PyTorch untuk tensor dan autograd

import torch.nn as nn
# Mengimpor modul neural network PyTorch

import numpy as np
# Mengimpor NumPy untuk manipulasi array

from sklearn import datasets
# Mengimpor dataset bawaan dari scikit-learn

from sklearn.preprocessing import StandardScaler
# Mengimpor StandardScaler untuk normalisasi fitur

from sklearn.model_selection import train_test_split
# Mengimpor fungsi untuk membagi data train dan test

In [ ]:
bc = datasets.load_breast_cancer()
# Memuat dataset breast cancer
# Dataset klasifikasi biner (jinak vs ganas)

X, y = bc.data, bc.target
# X: fitur (n_samples, n_features)
# y: label (0 atau 1)

n_samples, n_features = X.shape
# Mengambil jumlah sampel dan jumlah fitur

# print(n_samples, n_features)
# (Opsional) Menampilkan ukuran dataset

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1234
)
# Membagi data menjadi:
# 80% training, 20% testing
# random_state menjaga hasil tetap konsisten

sc = StandardScaler()
# Membuat objek StandardScaler

X_train = sc.fit_transform(X_train)
# Menghitung mean & std dari data train
# Lalu menormalkan X_train

X_test = sc.transform(X_test)
# Menormalkan X_test menggunakan parameter dari X_train
# (PENTING agar tidak data leakage)

# Konversi NumPy array ke tensor PyTorch
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test  = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test  = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
# Mengubah shape label train menjadi (n_samples, 1)
# Agar sesuai dengan output model

y_test = y_test.view(y_test.shape[0], 1)
# Mengubah shape label test menjadi (n_samples, 1)

# f = wx + b, lalu dilewatkan ke sigmoid
# Ini adalah logistic regression

class LogisticRegression(nn.Module):
  # Mendefinisikan model Logistic Regression
  # dengan mewarisi nn.Module

  def __init__(self, n_input_features):
    super(LogisticRegression, self).__init__()
    # Memanggil constructor parent class

    self.linear = nn.Linear(n_input_features, 1)
    # Layer linear: wx + b

  def forward(self, x):
    y_predicted = torch.sigmoid(self.linear(x))
    # Mengaplikasikan sigmoid agar output berada di rentang [0,1]
    return y_predicted

model = LogisticRegression(n_features)
# Membuat instance model dengan jumlah fitur input

learning_rate = 0.01
# Learning rate untuk optimisasi

criterion = nn.BCELoss()
# Binary Cross Entropy Loss
# Cocok untuk klasifikasi biner dengan output sigmoid

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
# Optimizer Stochastic Gradient Descent

num_epochs = 100
# Jumlah epoch training

for epoch in range(num_epochs):
  # Loop training

  y_predicted = model(X_train)
  # Forward pass pada data training

  loss = criterion(y_predicted, y_train)
  # Menghitung loss antara prediksi dan label asli

  loss.backward()
  # Backpropagation: menghitung gradien semua parameter model

  optimizer.step()
  # Update bobot dan bias

  optimizer.zero_grad()
  # Mengosongkan gradien agar tidak terakumulasi

  if (epoch + 1) % 10 == 0:
    print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

with torch.no_grad():
  # Menonaktifkan autograd saat evaluasi

  y_predicted = model(X_test)
  # Prediksi pada data test

  y_predicted_cls = y_predicted.round()
  # Mengubah probabilitas menjadi kelas 0 atau 1

  acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
  # Menghitung akurasi:
  # jumlah prediksi benar / total data test

  print(f'accuracy = {acc:.4f}')

# RINGKASAN:
# Kode ini mengimplementasikan Logistic Regression lengkap menggunakan PyTorch:
# - Dataset klasifikasi biner (breast cancer)
# - Data dipisah train/test dan dinormalisasi
# - Model dibuat dengan nn.Linear + sigmoid
# - Loss function menggunakan Binary Cross Entropy
# - Training dilakukan dengan SGD dan backpropagation
# - Evaluasi model menggunakan akurasi
# Ini adalah pipeline standar klasifikasi biner dalam deep learning.

epoch: 10, loss = 0.5202
epoch: 20, loss = 0.4321
epoch: 30, loss = 0.3751
epoch: 40, loss = 0.3352
epoch: 50, loss = 0.3057
epoch: 60, loss = 0.2828
epoch: 70, loss = 0.2645
epoch: 80, loss = 0.2495
epoch: 90, loss = 0.2369
epoch: 100, loss = 0.2262
accuracy = 0.9298
